In [1]:
#default_exp python_call_graph

In [2]:
#export

import csrgraph as cg
import pandas as pd
import numpy as np
from github_search import python_tokens, paperswithcode_tasks
from mlutil.feature_extraction import embeddings
from mlutil import prototype_selection
import mlutil
from mlutil.feature_extraction import embeddings 

import ast
import astunparse
import csrgraph
import nodevectors

import os
import itertools
import igraph
from sklearn import metrics
import gensim

from operator import itemgetter
import pickle

In [3]:
%cd ..

/home/kuba/Projects/github_search


In [4]:
python_files_df = pd.read_csv('data/python_files.csv')

In [5]:
python_files_df

,owner,repo_name,file_path,content,sha
0,trangvu,ape-npi,run-tests.py,#!/usr/bin/env python3\nimport subprocess\nimp...,5364c4a113de40cb17e40b052fa012ad7daba2bd
1,trangvu,ape-npi,scripts/apply_bpe.py,#!/usr/bin/env python3\n# Author: Rico Sennric...,261dd0ffc2e10ee32ea99ca80c99cde50a6e7636
2,trangvu,ape-npi,scripts/concat-bpe.py,#!/usr/bin/env python3\n\nimport argparse\npar...,e6bc04ce4b16c0b91d00fc576ed5d261c711760f
3,trangvu,ape-npi,scripts/copy-model.py,#!/usr/bin/env python3\nimport argparse\nimpor...,56014b1516fd6df606d500b5efc190efbe8d5203
4,trangvu,ape-npi,scripts/coverage.py,#!/usr/bin/env python3\n\nimport argparse\nfro...,f008bc3a194503c2e0c68ea2d5389db81905f6bd
...,...,...,...,...,...
560178,KyeongmoonKim,sb,tools/convert_from_depre.py,# --------------------------------------------...,4ed7125b8568a0f9c60a3bf4670747ec4c497942
560179,KyeongmoonKim,sb,tools/demo.py,#!/usr/bin/env python\n\n# -------------------...,2bd89335df588e010bbb22370274dcbd04bcb407
560180,KyeongmoonKim,sb,tools/reval.py,#!/usr/bin/env python\n\n# -------------------...,612ae317fa5362b2b03b667cc2a8e17323d1a16f
560181,KyeongmoonKim,sb,tools/test_net.py,# --------------------------------------------...,c9cfc53f46f7f9d6c0885ffcad8727f86716ce22


In [6]:
paperswithcode_df, all_papers_df = paperswithcode_tasks.get_paperswithcode_dfs()

In [7]:
papers_with_repo_df = paperswithcode_tasks.get_papers_with_biggest_tasks_df(10000)

/home/kuba/Projects/github_search/github_search/paperswithcode_tasks.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  papers_with_repo_with_biggest_tasks_df['most_common_task'] = papers_with_repo_with_biggest_tasks_df['tasks'].apply(


In [8]:
papers_with_repo_df['repo_name'] = papers_with_repo_df['repo'].str.split("/").apply(lambda ps: ps[1])

In [9]:
papers_with_repo_df

,paper_url,arxiv_id,title,abstract,url_abs,url_pdf,proceeding,authors,tasks,date,...,mentioned_in_github,mentioned_in_paper,paper_arxiv_id,paper_title,paper_url_abs,paper_url_pdf,repo,repo_url,most_common_task,repo_name
2,https://paperswithcode.com/paper/procedural-3d...,2010.06411,Procedural 3D Terrain Generation using Generat...,Procedural 3D Terrain generation has become a ...,https://arxiv.org/abs/2010.06411v1,https://arxiv.org/pdf/2010.06411v1.pdf,None,"[Emmanouil Panagiotou, Eleni Charou]","[Colorization, Image-to-Image Translation]",2020-10-13,...,False,True,2010.06411,Procedural 3D Terrain Generation using Generat...,https://arxiv.org/abs/2010.06411v1,https://arxiv.org/pdf/2010.06411v1.pdf,Panagiotou/Procedural3DTerrain,https://github.com/Panagiotou/Procedural3DTerrain,Image-to-Image Translation,Procedural3DTerrain
6,https://paperswithcode.com/paper/multiscale-sp...,2003.11265,Multiscale Sparsifying Transform Learning for ...,The data-driven sparse methods such as synthes...,https://arxiv.org/abs/2003.11265v4,https://arxiv.org/pdf/2003.11265v4.pdf,None,"[Ashkan Abbasi, Amirhassan Monadjemi, Leyuan F...","[Denoising, Dictionary Learning, Image Denoising]",2020-03-25,...,True,False,2003.11265,Multiscale Sparsifying Transform Learning for ...,https://arxiv.org/abs/2003.11265v4,https://arxiv.org/pdf/2003.11265v4.pdf,ashkan-abbasi66/MTLD,https://github.com/ashkan-abbasi66/MTLD,Denoising,MTLD
7,https://paperswithcode.com/paper/word2vec-opti...,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,Word2Vec is a prominent model for natural lang...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,None,"[Tosin P. Adewumi, Foteini Liwicki, Marcus Liw...","[Named Entity Recognition, Sentiment Analysis]",2020-03-23,...,True,False,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,phnk/D7047E,https://github.com/phnk/D7047E,Sentiment Analysis,D7047E
8,https://paperswithcode.com/paper/word2vec-opti...,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,Word2Vec is a prominent model for natural lang...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,None,"[Tosin P. Adewumi, Foteini Liwicki, Marcus Liw...","[Named Entity Recognition, Sentiment Analysis]",2020-03-23,...,False,True,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,tosingithub/sdesk,https://github.com/tosingithub/sdesk,Sentiment Analysis,sdesk
9,https://paperswithcode.com/paper/evaluating-se...,1903.07377,Evaluating Sequence-to-Sequence Models for Han...,Encoder-decoder models have become an effectiv...,https://arxiv.org/abs/1903.07377v2,https://arxiv.org/pdf/1903.07377v2.pdf,None,"[Johannes Michael, Roger Labahn, Tobias Grünin...","[Image Captioning, Keyword Spotting, Language ...",2019-03-18,...,True,False,1903.07377,Evaluating Sequence-to-Sequence Models for Han...,https://arxiv.org/abs/1903.07377v2,https://arxiv.org/pdf/1903.07377v2.pdf,jamestjw/OCR,https://github.com/jamestjw/OCR,Language Modelling,OCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74224,https://paperswithcode.com/paper/arbitrary-sty...,1703.06868,Arbitrary Style Transfer in Real-time with Ada...,Gatys et al. recently introduced a neural algo...,http://arxiv.org/abs/1703.06868v2,http://arxiv.org/pdf/1703.06868v2.pdf,ICCV 2017 10,"[Xun Huang, Serge Belongie]",[Style Transfer],2017-03-20,...,True,False,1703.06868,Arbitrary Style Transfer in Real-time with Ada...,http://arxiv.org/abs/1703.06868v2,http://arxiv.org/pdf/1703.06868v2.pdf,bethgelab/stylize-datasets,https://github.com/bethgelab/stylize-datasets,Style Transfer,stylize-datasets
74225,https://paperswithcode.com/paper/arbitrary-sty...,1703.06868,Arbitrary Style Transfer in Real-time with Ada...,Gatys et al. recently introduced a neural algo...,http://arxiv.org/abs/1703.06868v

In [10]:
python_files_df

,owner,repo_name,file_path,content,sha
0,trangvu,ape-npi,run-tests.py,#!/usr/bin/env python3\nimport subprocess\nimp...,5364c4a113de40cb17e40b052fa012ad7daba2bd
1,trangvu,ape-npi,scripts/apply_bpe.py,#!/usr/bin/env python3\n# Author: Rico Sennric...,261dd0ffc2e10ee32ea99ca80c99cde50a6e7636
2,trangvu,ape-npi,scripts/concat-bpe.py,#!/usr/bin/env python3\n\nimport argparse\npar...,e6bc04ce4b16c0b91d00fc576ed5d261c711760f
3,trangvu,ape-npi,scripts/copy-model.py,#!/usr/bin/env python3\nimport argparse\nimpor...,56014b1516fd6df606d500b5efc190efbe8d5203
4,trangvu,ape-npi,scripts/coverage.py,#!/usr/bin/env python3\n\nimport argparse\nfro...,f008bc3a194503c2e0c68ea2d5389db81905f6bd
...,...,...,...,...,...
560178,KyeongmoonKim,sb,tools/convert_from_depre.py,# --------------------------------------------...,4ed7125b8568a0f9c60a3bf4670747ec4c497942
560179,KyeongmoonKim,sb,tools/demo.py,#!/usr/bin/env python\n\n# -------------------...,2bd89335df588e010bbb22370274dcbd04bcb407
560180,KyeongmoonKim,sb,tools/reval.py,#!/usr/bin/env python\n\n# -------------------...,612ae317fa5362b2b03b667cc2a8e17323d1a16f
560181,KyeongmoonKim,sb,tools/test_net.py,# --------------------------------------------...,c9cfc53f46f7f9d6c0885ffcad8727f86716ce22


In [11]:
python_files_df = python_files_df.merge(papers_with_repo_df[['repo_name', 'tasks']], on=['repo_name'])

In [12]:
python_files_df

,owner,repo_name,file_path,content,sha,tasks
0,trangvu,ape-npi,run-tests.py,#!/usr/bin/env python3\nimport subprocess\nimp...,5364c4a113de40cb17e40b052fa012ad7daba2bd,"[Automatic Post-Editing, Machine Translation]"
1,trangvu,ape-npi,scripts/apply_bpe.py,#!/usr/bin/env python3\n# Author: Rico Sennric...,261dd0ffc2e10ee32ea99ca80c99cde50a6e7636,"[Automatic Post-Editing, Machine Translation]"
2,trangvu,ape-npi,scripts/concat-bpe.py,#!/usr/bin/env python3\n\nimport argparse\npar...,e6bc04ce4b16c0b91d00fc576ed5d261c711760f,"[Automatic Post-Editing, Machine Translation]"
3,trangvu,ape-npi,scripts/copy-model.py,#!/usr/bin/env python3\nimport argparse\nimpor...,56014b1516fd6df606d500b5efc190efbe8d5203,"[Automatic Post-Editing, Machine Translation]"
4,trangvu,ape-npi,scripts/coverage.py,#!/usr/bin/env python3\n\nimport argparse\nfro...,f008bc3a194503c2e0c68ea2d5389db81905f6bd,"[Automatic Post-Editing, Machine Translation]"
...,...,...,...,...,...,...
39906459,KyeongmoonKim,sb,tools/test_net.py,# --------------------------------------------...,c9cfc53f46f7f9d6c0885ffcad8727f86716ce22,[Object Detection]
39906460,KyeongmoonKim,sb,tools/trainval_net.py,# --------------------------------------------...,e7b4dab93b6e543402f25f53696e918f3e3d0b5a,[Object Detection]
39906461,KyeongmoonKim,sb,tools/trainval_net.py,# --------------------------------------------...,e7b4dab93b6e543402f25f53696e918f3e3d0b5a,[Object Detection]
39906462,KyeongmoonKim,sb,tools/trainval_net.py,# --------------------------------------------...,e7b4dab93b6e543402f25f53696e918f3e3d0b5a,"[Object Detection, Real-Time Object Detection,..."


In [17]:
%load_ext autoreload

In [18]:
code = """
def foo():
    bar()
"""

In [19]:
function_ast = (ast.parse(code).body[0])

In [20]:
ast.parse(code).body[0].name

'foo'

In [21]:
exprs = ast.parse(code).body[0].body

In [22]:
expr = exprs[0]

In [23]:
expr_ast = ast.parse("word = tuple(orig) + ''").body[0]

In [24]:
expr_ast.value

In [25]:
#export

def get_calls_from_expr_or_assign(expr):
    try:
        if type(expr) is ast.Name:
            return [expr.id]
        if type(expr) is ast.Call:
            return get_calls_from_expr_or_assign(expr.func) + get_calls_from_expr_or_assign(expr.args)
        elif type(expr) is ast.Attribute:
            return [expr.attr]
        elif type(expr) is ast.BinOp:
            return get_calls_from_expr_or_assign(expr.left) + get_calls_from_expr_or_assign(expr.right)
        elif type(expr) is ast.Expr:
            return get_calls_from_expr_or_assign(expr.value)
        elif type(expr) is ast.Assign:
            return get_calls_from_expr_or_assign(expr.value)
        elif type(expr) is ast.While or type(expr) is ast.If:
            return (
                get_calls_from_expr_or_assign(expr.test) +
                get_calls_from_expr_or_assign(expr.orelse) +
                get_calls_from_expr_or_assign(expr.body)
            )
        elif type(expr) is ast.For:
            return (
                get_calls_from_expr_or_assign(expr.target) +
                get_calls_from_expr_or_assign(expr.body) +
                get_calls_from_expr_or_assign(expr.iter)
            )
        elif type(expr) is ast.Try:
            return get_calls_from_expr_or_assign(expr.body)
        elif type(expr) is list:
            return [res for subexpr in expr for res in get_calls_from_expr_or_assign(subexpr)]
        else:
            return []
    except Exception as e:
        print(str(astunparse.unparse(expr)), ' ', str(e))

In [31]:
encode_code = """def encode(self, orig):
    word = tuple(orig) + ('</w>',)
    pairs = get_pairs(word)

    while True:
        bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
        if bigram not in self.bpe_codes:
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # don't print end-of-word symbols
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>', ''),)

    return word
"""

In [32]:
encode_ast = ast.parse(encode_code)

In [33]:
assign_body = encode_ast.body[0].body[0]

In [36]:
class_def = ast.parse(code).body[0]

In [37]:
class_def.body

In [38]:
#export


def zip_dicts(dicts):
    res = {}
    for d in dicts:
        res = {**res, **d}
    return res


def get_ast_function_calls(code_ast, calls={}):
    if type(code_ast) is ast.FunctionDef:
        return {code_ast.name: frozenset(
            sum([get_calls_from_expr_or_assign(expr) for expr in code_ast.body], [])
        )}
    elif type(code_ast) is ast.ClassDef:
        return {code_ast.name: frozenset(
            sum([get_calls_from_expr_or_assign(expr) for expr in code_ast.body], [])
        )}
    elif hasattr(code_ast, 'body'):
        return zip_dicts([get_ast_function_calls(item) for item in code_ast.body])
    else: return {}
    
    
def get_function_calls(code):
    code_ast = ast.parse(code)
    return get_ast_function_calls(code_ast)

In [39]:
e = ast.parse(
"new_word.extend(word[i:j])"
).body[0]

In [40]:
get_ast_function_calls(encode_ast.body[0])

{'encode': frozenset({'append',
            'bigram',
            'endswith',
            'extend',
            'first',
            'get_pairs',
            'i',
            'index',
            'j',
            'min',
            'new_word',
            'orig',
            'pairs',
            'second',
            'tuple',
            'word'})}

In [41]:
get_ast_function_calls(ast.parse(encode_code).body[0])

{'encode': frozenset({'append',
            'bigram',
            'endswith',
            'extend',
            'first',
            'get_pairs',
            'i',
            'index',
            'j',
            'min',
            'new_word',
            'orig',
            'pairs',
            'second',
            'tuple',
            'word'})}

In [42]:
get_function_calls(code)

{'foo': frozenset({'bar'})}

In [43]:
assert get_function_calls(code) == {"foo": frozenset(["bar"])}

In [44]:
items = ast.parse(code).body

In [76]:
example_file = python_files_df['content'].iloc[1]
print(example_file)

#!/usr/bin/env python3
# Author: Rico Sennrich

"""Use operations learned with learn_bpe.py to encode a new text.
The text will not be smaller, but use only a fixed vocabulary, with rare words
encoded as variable-length sequences of subword units.

Reference:
Rico Sennrich, Barry Haddow and Alexandra Birch (2016). Neural Machine Translation of Rare Words with Subword Units.
Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (ACL 2016). Berlin, Germany.
"""

import sys
import codecs
import argparse
import functools
from collections import defaultdict

class BPE(object):
    def encode(self, orig):
        """Encode word based on list of BPE merge operations, which are applied consecutively
        """
        word = tuple(orig) + ('</w>',)
        pairs = get_pairs(word)

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
            if bigram not in self.bpe_codes:
                break
     

In [82]:
fn_code = ast.parse(example_file).body[-2]
print(astunparse.unparse(fn_code))



def get_pairs(word):
    'Return set of symbol pairs in a word.\n\n    word is represented as tuple of symbols (symbols being variable-length strings)\n    '
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs



In [79]:
get_function_calls(example_file)

{'BPE': frozenset(),
 'create_parser': frozenset({'ArgumentParser', 'add_argument'}),
 'get_pairs': frozenset({'add', 'char', 'set'})}

In [48]:
items = ast.parse("a.append(1)").body
expr = items[0].value

In [49]:
%%time
python_files_df['clean_content'] = python_files_df['content'].str.replace("429: Too Many Requests", "")

CPU times: user 49.7 s, sys: 178 ms, total: 49.8 s
Wall time: 49.8 s


In [50]:
python_files_df['clean_content'] = python_files_df['clean_content'].apply(lambda s: np.nan if s == "" else s)

In [51]:
#export


def get_sample_files_df(python_files_df, n_files=100):
    "sample n_files from each repo"
    return python_files_df.dropna().groupby('repo_name').apply(lambda df: df.iloc[:min(df.shape[0], 100)]).reset_index(drop=True)

In [52]:
sample_df = python_files_df.dropna().groupby('repo_name').apply(lambda df: df.iloc[:min(df.shape[0], 100)]).reset_index(drop=True)

In [53]:
example_files = sample_df['clean_content']
example_filenames = sample_df['file_path'].apply(os.path.basename).str.replace('.py', '')

In [54]:
sample_df.columns

Index(['owner', 'repo_name', 'file_path', 'content', 'sha', 'tasks',
       'clean_content'],
      dtype='object')

In [55]:
#export


def get_repo_task_edges(python_files_df):
    task_exploded_df = python_files_df[['repo_name', 'tasks']].explode('tasks')
    task_exploded_df['tasks'] = task_exploded_df['tasks'].str.replace("2D ", "").str.replace("3D ", "").str.replace("4D ", "").str.replace("6D ", "")
    return task_exploded_df.groupby("tasks").apply(lambda df: {df['tasks'].iloc[0]: frozenset(df['repo_name'].values)}).tolist()

In [56]:
sample_df.columns

Index(['owner', 'repo_name', 'file_path', 'content', 'sha', 'tasks',
       'clean_content'],
      dtype='object')

In [57]:
repo_task_edges = get_repo_task_edges(sample_df)

In [58]:
repo_task_edges[:10]

[{'': frozenset({'deepmind-research', 'pytorch-image-models'})},
 {'AMR Parsing': frozenset({'stog'})},
 {'AMR-to-Text Generation': frozenset({'OpenNMT-py'})},
 {'Abstractive Text Summarization': frozenset({'BERT_text_summarisation',
             'Chatbot',
             'CopyCat-abstractive-opinion-summarizer',
             'Pointer_Transformer_Generator',
             'bottom-up-summary',
             'fairseq',
             'google-research',
             'onnxt5',
             'politicalsynthesis',
             'qags',
             'shmoop-corpus',
             'swap-net',
             'teaser_generate',
             'tensor2tensor',
             'transformers'})},
 {'Abuse Detection': frozenset({'multi-task-offensive-language-detection'})},
 {'Abusive Language': frozenset({'emo2vec_wassa_paper',
             'hate-speech-detection'})},
 {'Accented Speech Recognition': frozenset({'DeepSpeech',
             'STT',
             'audio2score',
             'models'})},
 {'Acoustic Scen

In [59]:
#export


def try_run(f): 
    def _maybe_failed_f(args):
        try:
            return f(args)
        except:
            return None
    return _maybe_failed_f

In [60]:
#export


def get_upper_level_edges(upper_level_names, lower_level_edges):
    return [
        {
            file_name: 
            frozenset(
                [
                    function 
                    for function in lower_level_edges.keys()
                ])
        }
        for (file_name, lower_level_edges) in zip(upper_level_names, lower_level_edges)
        if type(lower_level_edges) is dict
    ]


def make_records(function_edges):
    return [{
        'calling_function': calling_fn, 
        'called_function': called_fn
    }
        for calling_fn in function_edges.keys()
        for called_fn in function_edges[calling_fn]
    ]

In [61]:
#export


example_node_names = [
    'train', 'fit', 'test', '__init__', 'DistilBertForMaskedLM',
    'tensorflow', 'transformers', 'mmdetection', 'Recommenders-movielens',
    'hate-speech-detection', 'variational-dropout-sparsifies-dnn']


def get_nodes(call_graph, node_names):
    selected_nodes = call_graph.names[call_graph.names.isin(node_names)]
    return selected_nodes


def get_node_embedding_model_and_results(node_embeddings, call_graph, example_nodes, selected_names=None, topk=10):
    if selected_names is None:
        selected_nodes = call_graph.names
    else:
        selected_nodes = get_node_indices(call_graph, selected_names)
        
    
    results = []
    for (i, node_name) in enumerate(example_nodes.values):
        distances = metrics.euclidean_distances(node_embeddings[example_nodes.index[[i]]], node_embeddings) 
        results.append((call_graph.names[np.argsort(distances)[0,:topk]].values))
    results_df = pd.DataFrame(results)
    results_df.index =  example_nodes.values
    return results_df.T

In [62]:
#export


def get_dependency_edges(python_files_df, content_column='clean_content'):
    sample_files_df = python_files_df.dropna().groupby('repo_name').apply(lambda df: df.iloc[:min(df.shape[0], 100)]).reset_index(drop=True)
    filenames = sample_files_df['file_path'].apply(os.path.basename).str.replace('.py', '')
    files = sample_files_df[content_column]
    function_edges = files.apply(try_run(get_function_calls)).dropna()
    function_edges = function_edges.to_list()
    filename_edges = get_upper_level_edges(filenames, function_edges)
    repo_edges_dict = filenames.groupby(sample_files_df['repo_name']).agg(lambda args: frozenset(args)).to_dict()
    raw_repo_edges = [{k: repo_edges_dict[k]} for k in repo_edges_dict]
    root_edges = [{
        '<ROOT>': [repo for edges in raw_repo_edges for repo in repo_edges_dict.keys()]
        }
    ]
    repo_task_edges = get_repo_task_edges(sample_files_df)
    all_edges = root_edges + repo_task_edges + raw_repo_edges +  filename_edges + function_edges
    return all_edges

In [63]:
all_edges = get_dependency_edges(sample_df)

In [64]:
%%time
records  = [ record
    for edges in all_edges 
    for record in make_records(edges)
]

CPU times: user 1.07 s, sys: 284 ms, total: 1.35 s
Wall time: 1.35 s


In [65]:
len(set(map(itemgetter('calling_function'), records)))

66812

In [66]:
#export

def make_call_df(records):
    call_records_df = pd.DataFrame.from_records(records)
    call_records_df.columns = ['source', 'destination'] 
    call_records_df = call_records_df[
        (call_records_df['source'].apply(len) > 0) &
        (call_records_df['source'] != 'null') &
        (call_records_df['destination'].apply(len) > 0) &
        (call_records_df['source'] != call_records_df['destination'])
    ]
    return call_records_df.drop_duplicates()

In [67]:
call_records_df = make_call_df(records)

In [68]:
call_records_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844725 entries, 0 to 10035947
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   source       844725 non-null  object
 1   destination  844725 non-null  object
dtypes: object(2)
memory usage: 19.3+ MB


In [69]:
call_records_df.to_csv('data/call_records.csv', index=None)

In [70]:
relations = list(call_records_df.iloc[:,[1,0]].itertuples(index=False, name=None))

In [71]:
#export

def make_igraph(call_records_df):
    vertices = list(set(call_records_df['source'].unique()).union(call_records_df['destination'].unique()))
    edges = [
        (source, destination)
        for (source, destination) in zip(
            call_records_df['source'].values,
            call_records_df['destination'].values
        )
    ]
    graph = igraph.Graph()
    graph.add_vertices(vertices)
    graph.add_edges(edges)
    return graph

In [72]:
graph = make_igraph(call_records_df)

In [73]:
pickle.dump(graph, open("data/call_igraph.pkl", 'wb'))

In [74]:
graph.vcount()

198517

## nodevectors & csrgraph

In [ ]:
call_graph = cg.read_edgelist('data/function_calls.csv', sep=',')
example_nodes = get_nodes(call_graph, example_node_names)
pickle.dump(call_graph, open('call_csrgraph.pkl', 'wb'))

In [ ]:
example_repo_node_names = [
    'fairseq', 'mmdetection', 'Recommenders-movielens',
]
example_node_names = [
    'train', 'fit', 'test', '__init__', 'DistilBertForMaskedLM',
    'tensorflow', 
    'hate-speech-detection', 'variational-dropout-sparsifies-dnn']



def get_nodes(call_graph, node_names):
    selected_nodes = call_graph.names[call_graph.names.isin(node_names)]
    return selected_nodes


example_nodes = get_nodes(call_graph, example_node_names)
example_repo_nodes = get_nodes(call_graph, example_repo_node_names)


def get_node_embedding_model_and_results(node_embeddings, call_graph, example_nodes, selected_names=None, topk=10):
    if selected_names is None:
        selected_nodes = call_graph.names
    else:
        selected_nodes = get_node_indices(call_graph, selected_names)
        
    
    results = []
    for (i, node_name) in enumerate(example_nodes.values):
        distances = metrics.euclidean_distances(node_embeddings[example_nodes.index[[i]]], node_embeddings) 
        results.append((call_graph.names[np.argsort(distances)[0,:topk]].values))
    results_df = pd.DataFrame(results)
    results_df.index =  example_nodes.values
    return results_df.T

In [ ]:
call_records_df[call_records_df['source'] == 'fairseq']

In [ ]:
call_records_df[call_records_df['destination'] == 'fairseq']

In [ ]:
call_records_df[call_records_df['source'] == '<ROOT>']

In [ ]:
call_graph.mat.diagonal().sum()

%%time
ggvec_model = nodevectors.GGVec(n_components=100, max_epoch=1000)
ggvec_embeddings = ggvec_model.fit_transform(call_graph)

In [ ]:
%%time
prone_model = nodevectors.ProNE(n_components=100, step=100)
prone_embeddings = prone_model.fit_transform(call_graph)

get_node_embedding_model_and_results(ggvec_embeddings, call_graph, example_repo_nodes)

In [ ]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes)

In [ ]:
from gensim.models import KeyedVectors
prone_kv = KeyedVectors(prone_embeddings.shape[1])
prone_kv.add(call_graph.names, prone_embeddings)

In [ ]:
prone_kv.save("data/prone_embeddings.bin")

In [ ]:
prone_kv.most_similar("tensorflow")

In [ ]:
[k for k in prone_kv.vocab.keys() if 'allenn' in k]

In [ ]:
len(list(repo_edges[0].values())[0])

In [ ]:
list(repo_edges[0].values())[0]

In [ ]:
call_graph.names[call_graph.names.str.contains('allennlp')]

In [ ]:
[k for item in repo_edges for (k, v) in item.items() if 'basic_allennlp' in v]

In [ ]:
prone_kv.most_similar("num_block_x")

In [ ]:
with open('prone_closest_repos.md', 'w') as f:
    f.write(get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes).to_markdown())

In [ ]:
with open('prone_closest_nodes.md', 'w') as f:
    f.write(get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes).to_markdown())

In [ ]:
!cat prone_closest_nodes.md

In [ ]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes).to_csv(open('prone_closest_nodes.md', 'w'))

In [ ]:
get_node_embedding_model_and_results(ggvec_embeddings, call_graph, example_nodes)

In [ ]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes)

In [ ]:
from gensim.models import poincare

In [ ]:
example_edges = list(call_records_df.itertuples(index=False, name=None))

In [ ]:
call_records_df.shape

In [ ]:
model = poincare.PoincareModel(example_edges, negative=2)

In [ ]:
%%time
model.train(epochs=2, batch_size=64)

In [ ]:
model.kv.descendants("tensorflow")

# Hyperbolic embeddings

In [ ]:
import torch

In [ ]:
%load_ext autoreload

In [ ]:
torch.model

## TODO

- expore call graph
- better call graph
- Node2

Exploring call graph

what is the problem?

Graph is not even connected

In [ ]:
i = 2